### Startup imports

In [1]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import slurm_manager
from autoqchem.helper_classes import slurm_status

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [2]:
sm=slurm_manager(user='jchleinitz', host='neel.chimie.ens.fr')

### Provide some smiles string(s)

and copy into them into ```my_smiles``` variable. It's a list of strings (a Python list can be of any size)

In [3]:
my_smiles=[
    "c1ccc(O)cc1",
    "c1ccc(F)cc1",
    "c1ccc(Cl)cc1"
]

For a list of smiles, you can also read them out from a csv file in this way, assumed file name ```smiles_strings.csv```, also assuming the strings are in the first column, and there are no headers

In [4]:
# my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
# my_smiles

### Initialize the molecules and generate conformations

We will use some ```for``` loops to loop over the smiles

In [5]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, max_num_conformers=1))

INFO:autoqchem.molecule:Initializing molecule with canonical smiles: Oc1ccccc1
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Initializing molecule with canonical smiles: Fc1ccccc1
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Initializing molecule with canonical smiles: Clc1ccccc1
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.


Draw some molecules

In [6]:
#mols[0].draw(2)

### Create gaussian jobs for each conformation

In [7]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

/Users/julesschleinitz/Library/Application Support/neel/C6H6O_4368


INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C6H6O_4368_conf_0
/Users/julesschleinitz/Library/Application Support/neel/C6H5F_f024


INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C6H5F_f024_conf_0
/Users/julesschleinitz/Library/Application Support/neel/C6H5Cl_5300


INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C6H5Cl_5300_conf_0


In [8]:
sm.get_job_stats(split_by_can=True)

status,created
can,
Clc1ccccc1,1
Fc1ccccc1,1
Oc1ccccc1,1


### Submit the jobs to the server 

if you are not connected, it will ask for password + Duo

In [9]:
# not available since I didn't manage to automatize the connection to hermite...

In [10]:
#sm.submit_jobs()

Check the status of the jobs, locally and remotely

In [17]:
#sm.get_job_stats(split_by_can=True)

In [12]:
#sm.squeue(summary=False)

### Retrieve jobs

If there are some finished jobs the log files will be checked for completion and downloaded locally.

In [18]:
sm.retrieve_jobs()

INFO:autoqchem.slurm_manager:There are no jobs submitted to cluster. Nothing to retrieve.


### Resubmit incomplete jobs

In [19]:
sm.resubmit_incomplete_jobs()

INFO:autoqchem.slurm_manager:There are no incomplete jobs to resubmit.


### Upload molecules to the database

In [24]:
sm.upload_done_molecules_to_db()

TypeError: upload_done_molecules_to_db() missing 1 required positional argument: 'tags'

In [23]:
sm.get_jobs(slurm_status)

AttributeError: value